<a href="https://colab.research.google.com/github/ishitabera/NAISSProjects/blob/main/digit_classifier_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:00<00:00, 51815810.87it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 1726194.43it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:00<00:00, 14505170.84it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 3001501.30it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [ ]:
from torch.utils.data import DataLoader

loaders = {

           'train' : DataLoader(train_data,
                                 batch_size=100,
                                 shuffle=True,
                                 num_workers=1),

           'test'  : DataLoader(test_data,
                                 batch_size=100,
                                 shuffle=True,
                                 num_workers=1),
}

In [ ]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7a81762c3670>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7a81762c2530>}

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)

    return F.log_softmax(x)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\tLoss: {loss.item():.6f}')

def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(loaders['test'].dataset)
        print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%)\n')

In [ ]:
for epoch in range(1, 11):
    train(epoch)
    test()

<ipython-input-15-e2d67740874a>:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.283668
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.214358
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.777408
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.277416
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.079154
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.985745
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.796443
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.981511
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.543908
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.641005
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.783575
Train Epoch: 1 [22000/60000 (37%)]	Loss: 0.600771
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.494014
Train Epoch: 1 [26000/60000 (43%)]	Loss: 0.555745
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.533935
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.347515
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.715957
Train Epoch: 1 [34000/60000 (57%)]	Loss: 0.333420
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.454781
Train Epoch: 1 [38000/60000 (63%)]	Loss: 0.437147
Train Epoch